In [196]:
import os
import pickle
import json
import random
import numpy as np
!pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from read_data import read_file

In [197]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [198]:
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = "encoded"

In [199]:
!pip install torch_geometric --user

In [200]:
import os

# Define the path to the folder containing your Verilog files
folder_path = 'encoded'

# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


385
<class 'str'>
encoded\adder11.txt


In [201]:
import os
import json
import re

def label_verilog_file(file_name):
    label_mapping = {
        'adder': 0, 'ALU': 1, 'and': 2, 'bcd': 3, 'comparator': 4, 'decoder': 5,
        'encoder': 6, 'mult': 7, 'mux': 8, 'nand': 9, 'nor': 10, 'not': 11, 'or': 12,
        'pe': 13, 'seg': 14, 'sub': 15, 'xnor': 16, 'xor': 17
    }
    pattern = r"([a-zA-Z]+)(\d+)?"
    match = re.match(pattern, file_name)
    if match:
        base_name = match.group(1)
        if base_name in label_mapping:
            return label_mapping[base_name]
    return None

def add_label_to_verilog_file(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, "r") as file:
            loaded_data = json.load(file)
            label = label_verilog_file(os.path.basename(file_path))
            if label is not None and [label] not in loaded_data:  # Check if label already exists
                loaded_data.append([label])  # Add label as the third list
                with open(file_path, "w") as file:
                    json.dump(loaded_data, file)
                return True
    return False

for file_path in verilog_files:
    if not add_label_to_verilog_file(file_path):
        print(f"Failed to label: {file_path}")


Failed to label: encoded\adder11.txt
Failed to label: encoded\adder12.txt
Failed to label: encoded\adder13.txt
Failed to label: encoded\adder14.txt
Failed to label: encoded\adder15.txt
Failed to label: encoded\adder16.txt
Failed to label: encoded\adder17.txt
Failed to label: encoded\adder18.txt
Failed to label: encoded\adder19.txt
Failed to label: encoded\adder2.txt
Failed to label: encoded\adder20.txt
Failed to label: encoded\adder5.txt
Failed to label: encoded\adder8.txt
Failed to label: encoded\ALU10.txt
Failed to label: encoded\ALU13.txt
Failed to label: encoded\ALU14.txt
Failed to label: encoded\ALU15.txt
Failed to label: encoded\ALU2.txt
Failed to label: encoded\ALU6.txt
Failed to label: encoded\ALU7.txt
Failed to label: encoded\ALU9.txt
Failed to label: encoded\and1.txt
Failed to label: encoded\and10.txt
Failed to label: encoded\and12.txt
Failed to label: encoded\and13.txt
Failed to label: encoded\and14.txt
Failed to label: encoded\and15.txt
Failed to label: encoded\and16.txt
Fa

In [202]:
from torch_geometric.data import Data

def extracting_attributes(verilog_file):
    try:
        if os.path.isfile(verilog_file):
            with open(verilog_file, "r") as file:
                loaded_data = json.load(file)
                nodes = loaded_data[0]
                edges = loaded_data[1]
                label = loaded_data[2]
                x = torch.tensor(nodes, dtype=torch.float)
                edge_index = torch.tensor(edges, dtype=torch.long)
                y = torch.tensor(label, dtype=torch.float)
                data = Data(x=x, edge_index=edge_index, y = y)
                return data
    except Exception as e:
        return None


In [203]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader

class VerilogDataset(Dataset):  # Using Dataset from torch_geometric
    def __init__(self, verilog_files):
        print(f"Loaded {len(verilog_files)} Verilog files.")
        self.verilog_files = verilog_files

    def __len__(self):
        return len(self.verilog_files)

    def __getitem__(self, idx):
        verilog_file = self.verilog_files[idx]
        data = extracting_attributes(verilog_file)
        return data

dataset = VerilogDataset(verilog_files)
len(dataset)

Loaded 385 Verilog files.


385

In [204]:
def graph_stat(dataset):
    """
    TODO: calculate the statistics of the ENZYMES dataset.
    
    Outputs:
        min_num_nodes: min number of nodes
        max_num_nodes: max number of nodes
        mean_num_nodes: average number of nodes
        min_num_edges: min number of edges
        max_num_edges: max number of edges
        mean_num_edges: average number of edges
    """
    nodes_edges = [(data.num_nodes, data.num_edges) for data in dataset]
    num_nodes, num_edges = list(list(zip(*nodes_edges))[0]), list(list(zip(*nodes_edges))[1])
    min_num_nodes = min(num_nodes)
    max_num_nodes = max(num_nodes)
    mean_num_nodes = np.mean(num_nodes)
    min_num_edges = min(num_edges)
    max_num_edges = max(num_edges)
    mean_num_edges = np.mean(num_edges)
    
    print(f"min_num_nodes: {min_num_nodes}")
    print(f"max_num_nodes: {max_num_nodes}")
    print(f"mean_num_nodes: {mean_num_nodes}")
    print(f"min_num_edges: {min_num_edges}")
    print(f"max_num_edges: {max_num_edges}")
    print(f"mean_num_edges: {mean_num_edges}")

graph_stat(dataset)

min_num_nodes: 3
max_num_nodes: 165
mean_num_nodes: 15.501298701298701
min_num_edges: 2
max_num_edges: 163
mean_num_edges: 14.275324675324676


In [205]:
# from torch_geometric.data import DataLoader

# loader = DataLoader(dataset, batch_size=32, shuffle=True)

# loader_iter = iter(loader)
# batch = next(loader_iter)
# print(batch)
# print(batch.num_graphs)



# # split
# split_idx = int(len(dataset) * 0.8)
# train_dataset = dataset[:split_idx]
# test_dataset = dataset[split_idx:]

# print("len train:", len(train_dataset))
# print("len test:", len(test_dataset))


DataBatch(x=[637, 3], edge_index=[2, 593], y=[32], batch=[637], ptr=[33])
32


TypeError: object of type 'NoneType' has no len()

In [211]:
# from torch.utils.data import random_split

# # Define the sizes of training, validation, and test sets
# train_size = int(0.7 * len(dataset))  # 70% of the data for training
# val_size = int(0.15 * len(dataset))   # 15% of the data for validation
# test_size = len(dataset) - train_size - val_size  # Remaining data for testing

# # Split the dataset into training, validation, and test sets
# train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# # Create DataLoader for each set
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=32)
# test_dataloader = DataLoader(test_dataset, batch_size=32)


from torch.utils.data import random_split

# Define the size of the training set (e.g., 70% of the data)
train_size = int(0.7 * len(dataset))

# Calculate the size of the testing set
test_size = len(dataset) - train_size

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for each set
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)


In [212]:
import math
from torch_geometric.utils import to_dense_adj, add_self_loops

class GCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()
        self.theta = nn.Parameter(torch.FloatTensor(in_channels, out_channels))
        # Initialize the parameters.
        stdv = 1. / math.sqrt(out_channels)
        self.theta.data.uniform_(-stdv, stdv)
    
    def forward(self, x, edge_index):
        """
        TODO:
            1. Generate the adjacency matrix with self-loop \hat{A} using edge_index.
            2. Calculate the diagonal degree matrix \hat{D}.
            3. Calculate the output X' with torch.mm using the equation above.
        """

        num_nodes = x.shape[0]
        A = torch.sparse_coo_tensor(edge_index, torch.ones(edge_index.shape[1]), (num_nodes, num_nodes))
        A = A.to_dense()
        A_hat = A + torch.eye(num_nodes)
        
        A_sum = torch.sum(A_hat, dim=1)
        D = torch.pow(A_sum, -0.5)
        D[D == float('inf')] = 0.0
        D_hat_sqrt = torch.diag(D)
        
        first = torch.mm(torch.mm(D_hat_sqrt, A_hat), D_hat_sqrt)
        second = torch.mm(x, self.theta)
        
        ret = torch.mm(first, second)
        
        return ret

In [213]:
# from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        """
        TODO:
            1. Define the first convolution layer using `GCNConv()`. Set `out_channels` to 64;
            2. Define the first activation layer using `nn.ReLU()`;
            3. Define the second convolution layer using `GCNConv()`. Set `out_channels` to 64;
            4. Define the second activation layer using `nn.ReLU()`;
            5. Define the third convolution layer using `GCNConv()`. Set `out_channels` to 64;
            6. Define the dropout layer using `nn.Dropout()`;
            7. Define the linear layer using `nn.Linear()`. Set `output_size` to 2.

        Note that for MUTAG dataset, the number of node features is 7, and the number of classes is 2.

        """
        
        self.gcn1 = GCNConv(in_channels=7, out_channels=64)
        self.a1 = nn.ReLU()
        self.gcn2 = GCNConv(in_channels=64, out_channels=64)
        self.a2 = nn.ReLU()
        self.gcn3 = GCNConv(in_channels=64, out_channels=64)
        self.dropout = nn.Dropout(p=0.4)
        self.linear = nn.Linear(in_features=64, out_features=2)

    def forward(self, x, edge_index, batch):
        """
        TODO:
            1. Pass the data through the frst convolution layer;
            2. Pass the data through the activation layer;
            3. Pass the data through the second convolution layer;
            4. Obtain the graph embeddings using the readout layer with `global_mean_pool()`;
            5. Pass the graph embeddgins through the dropout layer;
            6. Pass the graph embeddings through the linear layer.
            
        Arguments:
            x: [num_nodes, 7], node features
            edge_index: [2, num_edges], edges
            batch: [num_nodes], batch assignment vector which maps each node to its 
                   respective graph in the batch

        Outputs:
            probs: probabilities of shape (batch_size, 2)
        """
        x = self.gcn1(x, edge_index)
        x = self.a1(x)
        x = self.gcn2(x, edge_index)
        x = self.a2(x)
        x = self.gcn3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.linear(x)
        
        probs = torch.nn.functional.softmax(x, dim=-1)
        
        return probs
        
        
        
GCN()

GCN(
  (gcn1): GCNConv()
  (a1): ReLU()
  (gcn2): GCNConv()
  (a2): ReLU()
  (gcn3): GCNConv()
  (dropout): Dropout(p=0.4, inplace=False)
  (linear): Linear(in_features=64, out_features=2, bias=True)
)

In [214]:
gcn = GCN()

# optimizer
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)
# loss
criterion = torch.nn.CrossEntropyLoss()

def train(train_loader):
    gcn.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        """
        TODO: train the model for one epoch.
        
        Note that you can acess the batch data using `data.x`, `data.edge_index`, `data.batch`, `data,y`.
        """
        
        out = gcn(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        
def test(loader):
    gcn.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = gcn(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(200):
    train(train_dataloader)
    train_acc = test(train_dataloader)
    test_acc = test(test_dataloader)
    print(f'Epoch: {epoch + 1:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (369x3 and 7x64)